### Authenticate and get headers

In [ ]:
import requests
from _includes import config
from _includes import helper

config_dict = config.config_dict

api = helper.APIHelper(config_dict)
base_site_url = f'{api.server}/api/{api.api_version}/sites/{api.site_id}'
base_api_url = f'{api.server}/api/-'
headers = api.headers

### Prepare to Create Definition

In [ ]:
datasource_dict_tuple = api.get_datasource_dict_tuple()
datasource_id = datasource_dict_tuple[('default','SuperStoreDataDev')]
print(datasource_id)

pulse_name = 'DataDev Demo 02'
pulse_description = 'Number of Sales Amount - Demo 02'

### Measure Basic

In [ ]:
# AGGREGATIONS
# ---------------------------------------------------------------------
# AGGREGATION_SUM / AGGREGATION_AVERAGE / AGGREGATION_MEDIAN
# AGGREGATION_MAX / AGGREGATION_MIN / AGGREGATION_COUNT / AGGREGATION_COUNT_DISTINCT
# Running Total can be True only for AGGREGATION_SUM

measure_field = 'Sales'
measure_aggregate = 'AGGREGATION_AVERAGE'
is_running_total = False

# Time Dimension and Filters
time_dimension_field = 'OrderDate'
list_allowed_dimension = ['Category','Sub-Category','Region','Country','Segment']

In [ ]:
# GRANULARITY_BY_YEAR / GRANULARITY_BY_QUARTER / GRANULARITY_BY_MONTH / GRANULARITY_BY_WEEK / GRANULARITY_BY_DAY
list_allowed_granularity = []

# NUMBER_FORMAT_TYPE_NUMBER / NUMBER_FORMAT_TYPE_PERCENT / NUMBER_FORMAT_TYPE_CURRENCY
number_format = 'NUMBER_FORMAT_TYPE_NUMBER'

# SENTIMENT_TYPE_NONE / SENTIMENT_TYPE_UP_IS_GOOD / SENTIMENT_TYPE_DOWN_IS_GOOD
sentiment_type = 'SENTIMENT_TYPE_UP_IS_GOOD'

# NOUNS
singular_noun = ''
plural_noun = ''

### Create a JSON payload

In [ ]:
payload = {
    'name': pulse_name,
    'description': pulse_description,
    'specification': {
        'datasource': {
            'id': datasource_id
        },
        'basic_specification': {
            'measure': {
                'field': measure_field,
                'aggregation': measure_aggregate
            },
            'time_dimension': {
                'field': time_dimension_field
            },
            'filters': []
        },
        'is_running_total': is_running_total
    },
    'extension_options': {
        'allowed_dimensions': list_allowed_dimension,
        'allowed_granularities': list_allowed_granularity,
        'offset_from_today': 0
    },
    'representation_options': {
        'type': number_format,
        'number_units': {
            'singular_noun': singular_noun,
            'plural_noun': plural_noun
        },
        'sentiment_type': sentiment_type
    },
    'comparisons': {
        'comparisons': [
            {
                'compare_config': {
                    'comparison': 'TIME_COMPARISON_PREVIOUS_PERIOD'
                },
                'index': 0
            },
            {
                'compare_config': {
                    'comparison': 'TIME_COMPARISON_YEAR_AGO_PERIOD'
                },
                'index': 1
            }
        ]
    }
}

### Post to Pulse Definitions Endpoint

In [ ]:
endpoint = f'{base_api_url}/pulse/definitions'

try:
    response = requests.post(endpoint, headers=headers, json=payload)
    response.raise_for_status()
    print('Metric definition created successfully.')
    j = response.json()
    definition_id = j['definition']['metadata']['id']
    print(f'Metric Definition ID:')
    print(definition_id)
except requests.exceptions.RequestException as e:
    print(f'Request failed: {e}')
    if response.content:
        print('Response content:', response.content.decode())